In [ ]:
import sys
import os

# Add the parent directory of 'utilities' to the Python path
sys.path.append(r'C:\Users\Ai Sukmoren\Desktop\chatbot')

from dotenv import load_dotenv
from utilities.llms import llm
import os
from langchain.prompts import PromptTemplate
from langchain_community.tools import YouTubeSearchTool
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from IPython.display import display, HTML
from utilities.llms import llm
import os
import io
import ast
import re
import ast


# Load the environment variables from .env file
load_dotenv()

# Now you can access these variables using os.getenv
url=os.getenv('NE04J_CONN_URL') 
username=os.getenv('NEO4J_USERNAME') 
password=os.getenv('NEO4J_PASSWORD')
database=os.getenv('NE04J_DB')

In [ ]:
import os
import sys
import io
import re
from neo4j import GraphDatabase
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate

def kg_gen(query):
    # Initialize the Neo4j graph connection
    graph = Neo4jGraph(url=os.getenv('NE04J_CONN_URL'), 
                        username=os.getenv('NEO4J_USERNAME'), 
                        password=os.getenv('NEO4J_PASSWORD'),
                        database=os.getenv('NE04J_DB'))

    # Define the prompt template
    prompt = """
    You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
    Convert the user's question based on the schema.

    instruction:
    - all animes is in lower cases
    - all nodes include Anime and Genre

    Varibales:
    Schema {schema},
    Question {question}
    """

    cypher_generation_prompt = PromptTemplate(
        template=prompt,
        input_variables=["schema", "question"],
    )

    # Create the GraphCypherQAChain instance
    cypher_chain = GraphCypherQAChain.from_llm(
        llm=llm,
        graph=graph,
        cypher_prompt=cypher_generation_prompt,
        verbose=True
    )


    # Capture the verbose output
    old_stdout = sys.stdout
    sys.stdout = io.StringIO()

    # Invoke the chain with the question
    res = cypher_chain.invoke(question)

    # Get the verbose output
    verbose_output = sys.stdout.getvalue()
    sys.stdout = old_stdout

    # Function to extract Full Context
    def extract_full_context(text):
        match = re.search(r'Full Context:\n\x1b\[32;1m\x1b\[1;3m(.*?)\x1b\[0m', text, re.DOTALL)
        if match:
            return match.group(1)
        return "Full Context not found."
    
    full_context = extract_full_context(verbose_output)
    
    return res,full_context

# Define the question
question = "What is the top 3 highest anime rating where rating is not null"

# Extract and print the Full Context
res,full_context = kg_gen(question)

print("question:")
print(res['query'])
print("Full Context:")
print(full_context)
print("Output:")
print(res['result'])

## G-eval

- **G-Eval Overview**: G-Eval is a framework developed to use large language models (LLMs) for evaluating LLM outputs, offering task-specific metrics with better human alignment. It uses chain of thoughts (CoTs) to generate evaluation steps and scores outputs from 1 to 5 based on these steps.

- **G-Eval Algorithm**: The algorithm involves introducing an evaluation task to an LLM, defining criteria, generating evaluation steps, creating a prompt, and generating a score. Optionally, probabilities of output tokens can be used for fine-grained scores and minimizing bias.

- **Performance**: G-Eval outperforms traditional, non-LLM evaluations by achieving higher alignment with human judgment, demonstrated through higher Spearman and Kendall-Tau correlations. Despite its accuracy, it can still be arbitrary as it relies on LLMs for scoring.

In [ ]:
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval.metrics import GEval
import nest_asyncio
# Apply nest_asyncio patch
nest_asyncio.apply()

# Assuming input and output are defined elsewhere in your code
input = "Your input text here"
output = "Your output text here"

# Create a test case
test_case = LLMTestCase(input=input, actual_output=output)

# Define the coherence metric
coherence_metric = GEval(
    name="Coherence",
    criteria="Coherence - the collective quality of all sentences in the actual output",
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
)

# Measure the coherence
coherence_metric.measure(test_case)

# Normalize the score if necessary
# Assuming the score should be between 1 and 5
normalized_score = coherence_metric.score * 4 + 1

# Print the results
print("Raw score:", coherence_metric.score)
print("Normalized score (1-5 range):", normalized_score)
print("Reason:", coherence_metric.reason)


##  Faithfulness

- **Faithfulness Metric**: Evaluates if the LLM's outputs in a RAG pipeline factually align with the information in the retrieval context. The QAG Scorer is recommended for this metric.

- **QAG Scorer Algorithm**: Extract all claims from the LLM output, verify each claim against the retrieval context nodes, and use closed-ended questions to check agreement. Calculate faithfulness by dividing the number of truthful claims ('yes' and 'idk') by the total number of claims.

- **Accuracy and Reliability**: This method leverages LLM's reasoning capabilities and ensures accurate evaluation, avoiding the unreliability of LLM-generated scores, making it superior to methods like G-Eval.


In [ ]:
# Define the question
question = "What is the top 3 highest anime rating where rating is not null"
res = cypher_chain.invoke(question)
full_context = extract_full_context(verbose_output)

from deepeval.metrics import FaithfulnessMetric
from deepeval.test_case import LLMTestCase

test_case=LLMTestCase(
  input=question, 
  actual_output=res['result'],
  retrieval_context=[full_context]
)
metric = FaithfulnessMetric(threshold=0.5)

metric.measure(test_case)
print(metric.score)
print(metric.reason)
print(metric.is_successful())

## Answer Relevancy

- **Answer Relevancy Metric**: This RAG metric assesses if the generator outputs concise and relevant answers by calculating the proportion of relevant sentences in the LLM output.

- **Calculation Method**: Determine answer relevancy by dividing the number of relevant sentences by the total number of sentences, considering the retrieval context to justify relevancy.

- **Implementation**: Utilize the `AnswerRelevancyMetric` from the `deepeval.metrics` library to build a robust answer relevancy evaluation, incorporating additional context as needed.


In [ ]:
# Define the question
question = "What is the top 3 highest anime rating where rating is not null"
res = cypher_chain.invoke(question)
full_context = extract_full_context(verbose_output)

from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase

test_case=LLMTestCase(
  input=question, 
  actual_output=res['result'],
  retrieval_context=[full_context]
)
metric = AnswerRelevancyMetric(threshold=0.5)

metric.measure(test_case)
print(metric.score)
print(metric.reason)
print(metric.is_successful())

## Contextual Relevancy

- **Contextual Relevancy Metric**: Measures the proportion of sentences in the retrieval context that are relevant to the given input.

- **Calculation Method**: Determine contextual relevancy by dividing the number of relevant sentences in the retrieval context by the total number of sentences.

- **Simplicity and Clarity**: This metric provides a straightforward way to assess the alignment of retrieval context with the input, ensuring concise and pertinent information retrieval.


In [ ]:
# Define the question
question = "What is the top 3 highest anime rating where rating is not null"
res = cypher_chain.invoke(question)
full_context = extract_full_context(verbose_output)

from deepeval.metrics import ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase

test_case=LLMTestCase(
  input=question, 
  actual_output=res['result'],
  retrieval_context=[full_context]
)
metric = ContextualRelevancyMetric(threshold=0.5)

metric.measure(test_case)
print(metric.score)
print(metric.reason)
print(metric.is_successful())